In [115]:
#getting our own spotify data

In [116]:
#importing libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import time
import datetime
import os
import csv
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing


In [117]:
#Authentication - without user using Spotify's Web API and making a new spotify developer account 
client_credentials_manager = SpotifyClientCredentials(client_id="14ab08163a0649be865dc476a3076fd7", client_secret="c33964ff56e24f868580c3d3c488304c")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [118]:
#Getting user playlist
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"]
              for x in sp.playlist_tracks(playlist_URI)["items"]]


In [119]:
#Acosticness confidence measure from 0.0 to 1.0 of whether the track is acoustic.
#Danceability measure describes how suitable a track is for dancing.
#duration_ms is the duration of the song track in milliseconds.
#Energy represents a perceptual measure of intensity and activity.
#Instrumentalness predicts whether a track contains vocals or not.
#Loudness of a track in decibels(dB).
#Liveness detects the presence of an audience in the recording.
#Speechiness detects the presence of spoken words in a track
#Time_signature is an estimated overall time signature of a track.
#Key the track is in. Integers map to pitches using standard Pitch Class notation.
#Valence measures from 0.0 to 1.0 describing the musical positiveness conveyed by a track.
#Target value describes the encoded value of 0 and 1. 0 means listener has not saved the song and 1 means listener have saved the song.
#Tempo is in beats per minute(BPM).
#Mode indicates the modality(major or minor) of the song.
#Song_title is the name of the song.
#Artist is the singer of the song.


In [120]:
#making a new csv file with th data and headers as the features of the audio features and also Song Name and Artist Name
song_name = []
artist_name = []
acousticness = []
danceability = []
duration_ms = []
energy = []
instrumentalness = []
loudness = []
liveness=[]
speechiness = []
time_signature = []
key = []
valence = []
tempo = []
mode = []
target = []
#get track name and artist name
for track_uri in track_uris:
    track = sp.track(track_uri)
    song_name.append(track["name"])
    artist_name.append(track["artists"][0]["name"])
    audio_features = sp.audio_features(track_uri)
    acousticness.append(audio_features[0]["acousticness"])
    danceability.append(audio_features[0]["danceability"])
    duration_ms.append(audio_features[0]["duration_ms"])
    energy.append(audio_features[0]["energy"])
    instrumentalness.append(audio_features[0]["instrumentalness"])
    loudness.append(audio_features[0]["loudness"])
    liveness.append(audio_features[0]["liveness"])
    speechiness.append(audio_features[0]["speechiness"])
    time_signature.append(audio_features[0]["time_signature"])
    key.append(audio_features[0]["key"])
    valence.append(audio_features[0]["valence"])
    tempo.append(audio_features[0]["tempo"])
    mode.append(audio_features[0]["mode"])
    target.append(1)
#make a csv file with the data
df = pd.DataFrame({'Song Name':song_name, 'Artist Name':artist_name, 'Acousticness':acousticness, 'Danceability':danceability, 'Duration_ms':duration_ms, 'Energy':energy, 'Instrumentalness':instrumentalness, 'Loudness':loudness, 'Speechiness':speechiness, 'Time_signature':time_signature, 'Key':key, 'Valence':valence, 'Tempo':tempo, 'Mode':mode, 'Target':target})
df.to_csv('audio_features.csv', index=False)


In [ ]:
df = pd.read_csv("Audio_features.csv")
df.head()

,Song Name,Artist Name,Acousticness,Danceability,Duration_ms,Energy,Instrumentalness,Loudness,Speechiness,Time_signature,Key,Valence,Tempo,Mode,Target
0,As It Was,Harry Styles,0.3420,0.520,167303,0.731,0.001010,-5.338,0.0557,4,6,0.662,173.930,0,1
1,Moscow Mule,Bad Bunny,0.2940,0.804,245940,0.674,0.000001,-5.453,0.0333,4,5,0.292,99.968,0,1
2,Me Porto Bonito,Bad Bunny,0.0901,0.911,178567,0.712,0.000027,-5.105,0.0817,4,1,0.425,92.005,0,1
3,Ojitos Lindos,Bad Bunny,0.0800,0.647,258299,0.686,0.000001,-5.745,0.0413,4,3,0.268,79.928,0,1
4,Tití Me Preguntó,Bad Bunny,0.0993,0.650,243717,0.715,0.000291,-5.198,0.2530,4,5,0.187,106.672,0,1


In [ ]:
from sklearn.preprocessing import MinMaxScaler
feature_cols = ['Acousticness', 'Danceability', 'Duration_ms', 'Energy',
                'Instrumentalness', 'Key', 'liveness', 'loudness', 'mode',
                'speechiness', 'tempo', 'time_signature', 'valence', ]

scaler = MinMaxScaler()
normalized_df = scaler.fit_transform(df[feature_cols])

print(normalized_df[:2])


KeyError: "None of [Index(['acousticness', 'danceability', 'duration_ms', 'energy',\n       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',\n       'speechiness', 'tempo', 'time_signature', 'valence'],\n      dtype='object')] are in the [columns]"